# **Scraping FC Barcelona Stats from fbref website**

In [ ]:
!pip install playwright

In [ ]:
!playwright install

In [ ]:
!pip install fake_useragent

In [4]:
import asyncio
from playwright.async_api import async_playwright
from fake_useragent import UserAgent
import pandas as pd
import time, random
import os

In [ ]:
ua = UserAgent()

async def scrape_fbref():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.set_extra_http_headers({"User-Agent": ua.random})
        try:
            print("Scraping fbref")
            await page.goto("https://fbref.com/en/squads/206d90db/2024-2025/c8/Barcelona-Stats-Champions-League", timeout=30000)
            await page.wait_for_selector(".table_wrapper", timeout=30000)
            tables = await page.query_selector_all(".table_wrapper")
            # Folder for CSVs
            os.makedirs("fbref_tables", exist_ok=True)

            for i, wrapper in enumerate(tables):
                try:
                    # Table title for CSV filename
                    title_elem = await wrapper.query_selector("h2")
                    title = await title_elem.inner_text() if title_elem else f"table_{i}"
                    title = title.strip().replace(" ", "_").replace("/", "-")

                    # Table element
                    table = await wrapper.query_selector("table")
                    if not table:
                        continue

                    # Headers
                    header_elems = await table.query_selector_all("thead tr:nth-child(2) th")
                    if not header_elems:
                        header_elems = await table.query_selector_all("thead tr:nth-child(1) th")

                    headers = []
                    for h in header_elems:
                        aria_label = await h.get_attribute("aria-label")
                        text = await h.inner_text()
                        headers.append(aria_label.strip() if aria_label else text.strip())

                    # Rows
                    row_elements = await table.query_selector_all("tbody tr")
                    rows = []
                    for row in row_elements:
                      cell_elements = await row.query_selector_all("th, td")
                      row_data = [await cell.inner_text() for cell in cell_elements]

                      first_cell = row_data[0].strip() if row_data else ""

                      # Skip if first cell is empty or matches any header
                      if not first_cell or first_cell in headers:
                          continue

                      rows.append(row_data)


                    # Save to CSV
                    df = pd.DataFrame(rows, columns=headers[:len(rows[0])])
                    filename = f"fbref_tables/{title}.csv"
                    df.to_csv(filename, index=False)
                    print(f"Saved table: {filename}")
                except Exception as inner_e:
                    print(f"Error parsing table {i}: {inner_e}")

        except Exception as e:
            print(f"Error: {e}")
        await browser.close()

# Run
await scrape_fbref()

Scraping fbref
Saved table: fbref_tables/Standard_Stats_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Scores_&_Fixtures_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Goalkeeping_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Advanced_Goalkeeping_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Shooting_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Passing_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Pass_Types_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Goal_and_Shot_Creation_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Defensive_Actions_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Possession_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Playing_Time_2024-2025_Barcelona:_Champions_League.csv
Saved table: fbref_tables/Miscellaneous_Stats_2024-2025_Barcelona:_Champions_L